# 4.2
如图所示的二次三角形单元，试计算$\frac{\partial N_4}{\partial x}$和$\frac{\partial N_4}{\partial y}$在点$P(1.5,2.0)$的数值。

In [ ]:
import sympy
from sympy import Matrix, Rational, Symbol
from sympy.abc import x, y
from finite_element.interpolation import SurfaceExclude, Isoparametric
from finite_element.interpolation.utils import convert_coordinate_global_to_local_triangular
sympy.init_printing()

## 定义变量

In [2]:
L1 = Symbol('L_1')
L2 = Symbol('L_2')
L1, L2

## 局部坐标与整体坐标

In [3]:
global_coordinates = [
    [0, 0], [4, 2], [1, 4],
    [Rational(4, 2), Rational(2, 2)],
    [Rational(5, 2), Rational(6, 2)],
    [Rational(1, 2), Rational(4, 2)],
]
global_coordinates

In [4]:
local_coordinates = [
    [1, 0],
    [0, 1],
    [0, 0],
    [Rational(1,2), Rational(1,2)],
    [0, Rational(1,2)],
    [Rational(1,2), 0],
]
local_coordinates

## 划线法得到插值函数

In [5]:
surface_exclude = SurfaceExclude([L1,L2])
surface_exclude.add_surface(L1)
surface_exclude.add_surface(L2)
surface_exclude.add_surface(1-L1-L2)
interpolation_functions = [
    surface_exclude.get_interpolation_function(coordinate)
    for coordinate in local_coordinates
]
Matrix(interpolation_functions)

⎡        L₁         ⎤
⎢                   ⎥
⎢        L₂         ⎥
⎢                   ⎥
⎢   -L₁ - L₂ + 1    ⎥
⎢                   ⎥
⎢      4⋅L₁⋅L₂      ⎥
⎢                   ⎥
⎢-4⋅L₂⋅(L₁ + L₂ - 1)⎥
⎢                   ⎥
⎣-4⋅L₁⋅(L₁ + L₂ - 1)⎦

## 初始化等参单元

In [6]:
isoparametric = Isoparametric(
    local_axises=[L1, L2],
    global_axises=[x, y],
    interpolation_functions=interpolation_functions,
    coordinates=global_coordinates,
)

## 计算雅克比矩阵

In [7]:
isoparametric.jacobian

⎡ -4⋅L₁ - 4⋅L₂ + 1   -16⋅L₁ - 16⋅L₂ + 4 ⎤
⎢                                       ⎥
⎣-4⋅L₁ - 20⋅L₂ + 13  -16⋅L₁ - 24⋅L₂ + 10⎦

## 计算插值函数对整体坐标的导数

In [8]:
isoparametric.displacement_interpolation_function_diff_global

⎡                                                                        ⎛    
⎢       8⋅L₁ + 12⋅L₂ - 5              -2                 -1           -4⋅⎝8⋅L₁
⎢──────────────────────────────   ────────────   ───────────────────  ────────
⎢7⋅(4⋅L₂ - 3)⋅(4⋅L₁ + 4⋅L₂ - 1)   7⋅(4⋅L₂ - 3)   7⋅(4⋅L₁ + 4⋅L₂ - 1)    7⋅(4⋅L
⎢                                                                             
⎢                                                                        ⎛    
⎢     -(4⋅L₁ + 20⋅L₂ - 13)              1                 2            2⋅⎝4⋅L₁
⎢───────────────────────────────  ─────────────  ───────────────────   ───────
⎣14⋅(4⋅L₂ - 3)⋅(4⋅L₁ + 4⋅L₂ - 1)  14⋅(4⋅L₂ - 3)  7⋅(4⋅L₁ + 4⋅L₂ - 1)    7⋅(4⋅L

2               2       ⎞     ⎛    2                          2           ⎞   
  - 2⋅L₁ - 12⋅L₂  + 5⋅L₂⎠   4⋅⎝8⋅L₁  + 16⋅L₁⋅L₂ - 10⋅L₁ + 4⋅L₂  - 7⋅L₂ + 2⎠   
──────────────────────────  ───────────────────────────────────────────────   
₂ - 3)⋅(4⋅L₁ + 4⋅L₂ - 1)             7⋅(4⋅L₂ - 3)⋅(

## 将整体坐标转化为局部坐标

In [13]:
L1_value, L2_value, L3_value = convert_coordinate_global_to_local_triangular(
    global_coordinates[0], global_coordinates[1], global_coordinates[2],
    (Rational(3,2), 2)
)
L1_value, L2_value, L3_value

## 代入点位求值

In [12]:
isoparametric.displacement_interpolation_function_diff_global.subs({
    L1: Rational(5, 14),
    L2: Rational(2, 7),
})

⎡                      148    116   -20 ⎤
⎢-9/143  2/13   -1/11  ────   ────  ────⎥
⎢                      1001   1001   91 ⎥
⎢                                       ⎥
⎢ -41                  -219   153       ⎥
⎢ ────   -1/26  2/11   ─────  ────  5/91⎥
⎣ 286                   1001  1001      ⎦

$\frac{\partial N_4}{\partial x}=\frac{148}{1001}$；$\frac{\partial N_4}{\partial y}=-\frac{219}{1001}$。